<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

## Query by source name/position

This notebook demonstrates how to use the ``astroquery.eso`` module to search for and download science-ready **HAWK-I** near-infrared data from the ESO archive, specifically targeting **Sgr A**, the supermassive black hole at the center of the Milky Way. It guides users through constructing and executing the query to locate relevant **HAWK-I** observations, filtering results to identify processed data products, and downloading these datasets for further analysis, providing a streamlined approach to accessing ESO’s extensive observational resources programatically. 

<hr style="border:2px solid #0281c9"> </hr>

### Importing and basic usage of astroquery.eso 

In this example, we will create an instance of the ESO class using:

```python
from astroquery.eso import Eso # import the ESO module from astroquery
eso = Eso()  # create an instance of the ESO class
```

This instance will be used to demonstrate various queries to the ESO Archive, including searching for raw data and data products from a range of instruments and sources.

In [1]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [2]:
eso = Eso() # create an instance of the ESO class 
eso.maxrec = 1000

### Identify available collections (reduced data)

ESO provides access to processed or reduced data, which has undergone various calibration and analysis steps to make it more immediately useful for scientific research. These processed datasets are grouped into `collections`, each corresponding to a specific survey, instrument, or science case. The list of available collections can be obtained as follows:

In [3]:
collections = eso.list_collections() # list all collections
collections[:3] # print the first 3 collections

['081.C-0827', '092.A-0472', '096.B-0054']

In [4]:
'HAWKI' in collections # check if 'HAWKI' is in the collections

True

### Performing a Small Cone Search Around Sgr A*

To perform a small cone search of **10 arcseconds** around the source **Sgr A**, we first need to resolve the source name to obtain its **Right Ascension (RA)** and **Declination (Dec)** coordinates. This can be done using an online name resolver such as the ``SkyCoord.from_name()`` method from astropy.coordinates, which allows us to resolve object names into precise RA/Dec coordinates using online astronomical databases. This gives: 

```python
ra = 266.835
dec = -28.38528
```

In coordinate values of degrees in the ICRS (International Celestial Reference System) frame.

In [5]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

coords = SkyCoord.from_name('Sgr A*') # create a SkyCoord object from the name of the source
radius = 20 *u.arcmin # set the radius of the search to 20 arcminutes

We use `astroquery.eso` to search for observations of **Sgr A** taken with the **HAWKI** instrument on the **VLT (Very Large Telescope)**.

This search can also be accessed via the following: 

[tap-link]: https://archive.eso.org/tap_obs/sync?REQUEST=doQuery&amp;LANG=ADQL&amp;MAXREC=200&amp;FORMAT=txt&amp;QUERY=SELECT%20*%0Afrom%20ivoa.obscore%0Awhere%20intersects(s_region,%20circle(%27%27,%20266.835,%20-28.38528,%200.1666))=1%0AAND%20instrument_name%20=%20%27HAWKI%27

[sp-link]: https://archive.eso.org/scienceportal/home?data_release_date=*:2025-02-07&pos=266.41682,-29.00782&r=0.016667&fovcorners=266.800868,-29.226927,266.033328,-29.226929,266.034957,-28.787142,266.799236,-28.787141&ins_id=HAWKI&dp_type=IMAGE&sort=dist,-fov,-obs_date&s=P%2fDSS2%2fcolor&f=0.669626&fc=266.799236,-28.787141&cs=J2000&av=true&ac=false&c=9,10,11,12,13,15,16,17,18,19,20,21&ta=RES&dts=true&sdtm=%7b%22IMAGE%22%3atrue%7d&at=266.41682,-29.00782&sr=i

- **TAP** - [see here][tap-link]
- **ESO Science Portal** [see here](sp-link)

In [6]:
instrument = 'HAWKI'       # set the instrument to HAWKI
ra = coords.ra             # get the right ascension of the source
dec = coords.dec           # get the declination of the source
radius = radius.to('deg')  # convert the radius to degrees

table = eso.query_collections(instrument, ra=ra.value, dec=dec.value, radius=radius.value) # query the ESO archive for VIRCAM data around Sgr A*
# table

In [7]:
table.sort('dp_id', reverse=True) # sort the table by the 'dp_id' column - newest data first

We can now inspect the results, which are stored as an ``astropy.table``. However note here these results include all data from the science achive, included non-science data... 

In [8]:
table[0] # print the first row of the table

abmaglim,access_estsize,access_format,access_url,bib_reference,calib_level,dataproduct_subtype,dataproduct_type,dp_id,em_max,em_min,em_res_power,em_xel,facility_name,filter,gal_lat,gal_lon,instrument_name,last_mod_date,multi_ob,n_obs,o_calib_status,o_ucd,obs_collection,obs_creator_did,obs_creator_name,obs_id,obs_publisher_did,obs_release_date,obs_title,obstech,p3orig,pol_states,pol_xel,preview_html,proposal_id,publication_date,release_description,s_dec,s_fov,s_pixel_scale,s_ra,s_region,s_resolution,s_xel1,s_xel2,snr,strehl,t_exptime,t_max,t_min,t_resolution,t_xel,target_name
mag,kbyte,,,,,,,,m,m,,,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,deg,deg,arcsec,deg,,arcsec,,,,,s,d,d,s,,
float64,int64,object,object,object,int32,object,object,object,float64,float64,float64,int64,object,object,float64,float64,object,object,object,int32,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,float64,float64,float64,float64,object,float64,int64,int64,float64,float64,float64,float64,float64,float64,int64,object
22.251,82751,application/x-votable+xml;content=datalink,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2021-12-16T10:45:26.076,,2,tile,image,ADP.2021-12-16T10:45:26.076,1.764e-06,1.474e-06,5.5828,1,ESO-VLT-U4,H,-0.001242,359.871789,HAWKI,2021-12-16T11:01:55.250Z,S,16,absolute,,HAWKI,ivo://eso.org/origfile?HI_SIMT_2355238_2019-08-21T02:32:20.829_f1H_f2OPEN_AO.fits,"ZOCCALI, MANUELA",2355238,ivo://eso.org/ID?ADP.2021-12-16T10:45:26.076,2020-08-21T03:00:44Z,,IMAGE,IDP,,--,https://archive.eso.org/dataset/ADP.2021-12-16T10:45:26.076,0103.B-0262(A),2021-12-16T11:01:55Z,http://www.eso.org/rm/api/v1/public/releaseDescriptions/87,-29.046277,0.19029199805,0.1066,266.32973,POLYGON J2000 266.435839 -29.025079 266.353596 -29.139129 266.223621 -29.067397 266.305951 -28.953426,0.299,4555,4555,--,--,480.0,58716.11688152,58716.10579663,957.734496,--,NPL066


We can check all the column names in the table by printing the ``table.colnames`` attribute to see what else we could filter... 

In [9]:
table[0].colnames # print the column names of the table

['abmaglim',
 'access_estsize',
 'access_format',
 'access_url',
 'bib_reference',
 'calib_level',
 'dataproduct_subtype',
 'dataproduct_type',
 'dp_id',
 'em_max',
 'em_min',
 'em_res_power',
 'em_xel',
 'facility_name',
 'filter',
 'gal_lat',
 'gal_lon',
 'instrument_name',
 'last_mod_date',
 'multi_ob',
 'n_obs',
 'o_calib_status',
 'o_ucd',
 'obs_collection',
 'obs_creator_did',
 'obs_creator_name',
 'obs_id',
 'obs_publisher_did',
 'obs_release_date',
 'obs_title',
 'obstech',
 'p3orig',
 'pol_states',
 'pol_xel',
 'preview_html',
 'proposal_id',
 'publication_date',
 'release_description',
 's_dec',
 's_fov',
 's_pixel_scale',
 's_ra',
 's_region',
 's_resolution',
 's_xel1',
 's_xel2',
 'snr',
 'strehl',
 't_exptime',
 't_max',
 't_min',
 't_resolution',
 't_xel',
 'target_name']

## Downloading datasets from the archive

The returned table has an ``dp_id`` column. It can be used to retrieve the datasets with ``retrieve_data()``.

The file names, returned in ``data_files``, points to the decompressed datasets (without the .Z extension) that have been locally downloaded. 

The default location (in the ``astropy`` cache) of the decompressed datasets can be adjusted by providing a destination keyword in the call to ``retrieve_data()``. By default, if a requested dataset is already found, it is not downloaded again from the archive. To force the retrieval of data that are present in the destination directory, use ``continuation=True`` in the call to ``retrieve_data()``.

In [10]:
data_files = eso.retrieve_data(table['dp_id'][0]) # download the first data product
# data_files = eso.retrieve_data(table['dp_id'][0], continuation=True) # force the download of all files even if present in the destination directory
# data_files = eso.retrieve_data(table['dp_id'][0], destination='./') # specify the destination directory for the download

INFO: Downloading datasets ... [astroquery.eso.core]
INFO: Downloading 1 files ... [astroquery.eso.core]
INFO: Downloading file 1/1 https://dataportal.eso.org/dataPortal/file/ADP.2021-12-16T10:45:26.076 to /Users/abarnes/.astropy/cache/astroquery/Eso [astroquery.eso.core]
INFO: Found cached file /Users/abarnes/.astropy/cache/astroquery/Eso/ADP.2021-12-16T10:45:26.076.fits [astroquery.eso.core]
INFO: Done! [astroquery.eso.core]


<hr style="border:2px solid #0281c9"> </hr>

## Check information about the downloaded file(s)

In [11]:
import webbrowser
import numpy as np
from astropy.table import MaskedColumn

def from_element_to_list(element, element_type=str):
    r"""Given an element it returns a list containing the element

    It also checks all the elements in the list have the same type defined by `element_type`

    Args:
        element (any): element that will be put in the list
        element_type (any): type of the element that should be contained in the list

    Returns:
        list: list containing `element`

    """
    if element is None:
        return None
    elif isinstance(element, list):
        for element_in_list in element:
            assert isinstance(element_in_list, element_type), r'{} must be a {}'.format(element_in_list, element_type)
        return element
    elif isinstance(element, np.ndarray):
        element_list: list = element.tolist()
        for element_in_list in element_list:
            assert isinstance(element_in_list, element_type), r'{} must be a {}'.format(element_in_list, element_type)
        return element_list
    elif isinstance(element, MaskedColumn):
        element_list = element.data.data.tolist()
        for element_in_list in element_list:
            assert isinstance(element_in_list, element_type), r'{} must be a {}'.format(element_in_list, element_type)
        return element_list
    elif isinstance(element, element_type):
        return [element]
    else:
        print('Not valid type for: {}'.format(element))
    return None

def open_url(url, show_link=True, open_link=True):
    r"""Open some url in the browser

    Args:
        url (str): url to open
        open_link (bool): open a link to page
        show_link (bool): show the link on the terminal

    Returns:
        None

    """
    if show_link:
        print(url)
    if open_link:
        webbrowser.open(url)
    return

def check_preview(dp_ids, show_link=False, open_link=True): 
    """Check the preview of the data product
        Note that this will download .pdf file to the browser download directory"""
    print('This will download a .pdf file from the browser directory')

    # Check if list
    if isinstance(dp_ids, str):
        dp_ids = [dp_ids]
    dp_ids = from_element_to_list(dp_ids, element_type=str)
    
    # Loop over the dp_ids
    for dp_id in dp_ids:
        url = f'https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?{dp_id}&eso_download=preview'
        open_url(url, show_link, open_link)


def check_header(dp_ids, show_link=False, open_link=True): 
    """Check the header of the data product"""
    print('This will open a new tab with the header information in browser')

    # Check if list
    if isinstance(dp_ids, str):
        dp_ids = [dp_ids]
    dp_ids = from_element_to_list(dp_ids, element_type=str)

    # Loop over the dp_ids
    for dp_id in dp_ids:
        url = f'https://archive.eso.org/hdr?DpId={dp_id}'
        open_url(url, show_link, open_link)


def check_scienceportal(dp_ids, show_link=False, open_link=True): 
    """Check the details of the data product"""
    print('This will open a new tab with the details in browser')

    # Check if list
    if isinstance(dp_ids, str):
        dp_ids = [dp_ids]
    dp_ids = from_element_to_list(dp_ids, element_type=str)

    # Loop over the dp_ids
    for dp_id in dp_ids:
        url = f'https://archive.eso.org/dataset/{dp_id}'
        open_url(url, show_link, open_link)


def check_hips(dp_ids, show_link=False, open_link=True):
    """Check the HIPS of the data product"""
    print('This will open a new tab with the HIPS information in browser')

    # Check if list
    if isinstance(dp_ids, str):
        dp_ids = [dp_ids]
    dp_ids = from_element_to_list(dp_ids, element_type=str)

    # Loop over the dp_ids
    for dp_id in dp_ids:
        url = f'https://archive.eso.org/previews/v1/files/{dp_id}/hips'
        open_url(url, show_link, open_link)


def check_release_info(dp_ids, show_link=False, open_link=True): 
    """Check the release info of the data product"""
    print('This will open a new tab with the release description infomation in browser')

    # Check if list
    if isinstance(dp_ids, str):
        dp_ids = [dp_ids]
    dp_ids = from_element_to_list(dp_ids, element_type=str)

    # Loop over the dp_ids
    for dp_id in dp_ids:
        url = f'https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?{dp_id}&eso_download=data_documentation'
        open_url(url, show_link, open_link)


def check_metadata(dp_ids, show_link=False, open_link=True):
    """Check the HTML metadata of the data product"""
    print('This will open a new tab with the metadata information in browser')

    # Check if list
    if isinstance(dp_ids, str):
        dp_ids = [dp_ids]
    dp_ids = from_element_to_list(dp_ids, element_type=str)

    # Loop over the dp_ids
    for dp_id in dp_ids:
        url = f'https://archive.eso.org/wdb/wdb/adp/phase3_main/query?dp_id={dp_id}'
        open_url(url, show_link, open_link)

# check_hips(table['dp_id'][0])
# check_header(table['dp_id'][0])
# check_scienceportal(table['dp_id'][0])
# check_release_info(table['dp_id'][0])
# check_metadata(table['dp_id'][0])
# check_preview(table['dp_id'][0])

---
## View is Aladin Lite
We now make use of the [ipyaladin](https://cds-astro.github.io/ipyaladin/getting_started/getting_started.html) package to view the data in [Aladin Lite](https://aladin.cds.unistra.fr/AladinLite/). 

The package should be easy to install through.

```bash
pip install ipyaladin
```


In [ ]:
from ipyaladin import Aladin # Aladin Lite widget for Jupyter notebooks
from astropy.io import fits # import the FITS module from astropy

We start by creating an Aladin Lite widget and setting the target position as the initial view. 

Here we show the [PanSTARRS](https://outerspace.stsci.edu/display/PANSTARRS/) survey data towards the central 0.2 degrees of the Galactic Centre (in galactic coordinates).

In [13]:
aladin = Aladin(fov=0.2, # Field of view in degrees
                survey='P/PanSTARRS/DR1/color-i-r-g', # Survey to display
                target='Sgr A*', # Target to display
                coo_frame="galactic") # Coordinate frame to use
aladin

Aladin(coo_frame='galactic', survey='P/PanSTARRS/DR1/color-i-r-g')

In [ ]:
hdu = fits.open(data_files)[0] # Open the FITS file downloaded from the ESO archive
aladin.add_fits(hdu, name='Sgr A*', opacity=0.25) # Add the FITS file to the Aladin Lite widget

<hr style="border:2px solid #0281c9"> </hr>